In [1]:
import tensorflow as tf

BATCH_SIZE = 100 
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99 

INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500

MODEL_SAVE_PATH='./deepNNModel/'
MODEL_NAME='deepNNModel'

import sys
#sys.path.append("./")
import os
import time
#from InferenceUtil import inference


class DeepNN:
    def __init__(self,batchSize=BATCH_SIZE,learningRateBase=LEARNING_RATE_BASE, 
                 inputNode=INPUT_NODE, outputNode=OUTPUT_NODE, layer1Node=LAYER1_NODE,
                 regularizationRate=REGULARIZATION_RATE, movingAverageDecay=MOVING_AVERAGE_DECAY,
                 learningRateDecay=LEARNING_RATE_DECAY,trainingSteps=TRAINING_STEPS,
                 modelSavePath=MODEL_SAVE_PATH, modelName=MODEL_NAME):
        self.batchSize = batchSize
        self.learningRateBase = learningRateBase
        self.inputNode=inputNode
        self.outputNode = outputNode
        self.layer1Node = layer1Node
        self.regularizationRate=regularizationRate
        self.movingAverageDecay=movingAverageDecay
        self.learningRateDecay=learningRateDecay
        self.trainingSteps=trainingSteps
        self.modelSavePath=modelSavePath
        self.modelName=modelName
    
    #also known as train
    def fit(self, trainData):
        tf.reset_default_graph() 
        x = tf.placeholder(tf.float32, [None, self.inputNode], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, self.outputNode], name='y-input')

        regularizer = tf.contrib.layers.l2_regularizer(self.regularizationRate)
        y = self.inference(x, regularizer)
        global_step = tf.Variable(0, trainable=False)

        variable_averages = tf.train.ExponentialMovingAverage(self.movingAverageDecay, global_step)
        variables_averages_op = variable_averages.apply(tf.trainable_variables())
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
        cross_entropy_mean = tf.reduce_mean(cross_entropy)
        loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
        learning_rate = tf.train.exponential_decay(
            self.learningRateBase,
            global_step,
            trainData.num_examples / self.batchSize, self.learningRateDecay,
            staircase=True)
        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
        with tf.control_dependencies([train_step, variables_averages_op]):
            train_op = tf.no_op(name='train')

        saver = tf.train.Saver()
        with tf.Session() as sess:
            tf.global_variables_initializer().run()

            for i in range(self.trainingSteps):
                xs, ys = trainData.next_batch(self.batchSize)
                _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
                if i % 1000 == 0:
                    print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
                    saver.save(sess, os.path.join(self.modelSavePath, self.modelName), global_step=global_step)
 
        return None
    
    def get_weight_variable(self, shape, regularizer):
        weights = tf.get_variable("weights", shape, initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(weights))
        return weights

    def inference(self, input_tensor, regularizer):
        with tf.variable_scope('layer1',reuse=tf.AUTO_REUSE):

            weights = self.get_weight_variable([self.inputNode, self.layer1Node], regularizer)
            biases = tf.get_variable("biases", [self.layer1Node], initializer=tf.constant_initializer(0.0))
            layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)

        with tf.variable_scope('layer2',reuse=tf.AUTO_REUSE):
            weights = self.get_weight_variable([self.layer1Node, self.outputNode], regularizer)
            biases = tf.get_variable("biases", [self.outputNode], initializer=tf.constant_initializer(0.0))
            layer2 = tf.matmul(layer1, weights) + biases

        return layer2

    #also known as train
    def predict(self, testData):
        with tf.Graph().as_default() as g:
            x = tf.placeholder(tf.float32, [None, self.inputNode], name='x-input')
            y_ = tf.placeholder(tf.float32, [None, self.outputNode], name='y-input')
            validate_feed = {x: testData.images, y_: testData.labels}

            y = self.inference(x, None)
            correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

            variable_averages = tf.train.ExponentialMovingAverage(self.movingAverageDecay)
            variables_to_restore = variable_averages.variables_to_restore()
            saver = tf.train.Saver(variables_to_restore)
            
            iCount = 10
            for iEachRound in range(0, iCount):
                with tf.Session() as sess:
                    ckpt = tf.train.get_checkpoint_state(self.modelSavePath)
                    if ckpt and ckpt.model_checkpoint_path:
                        saver.restore(sess, ckpt.model_checkpoint_path)
                        global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                        accuracy_score = sess.run(accuracy, feed_dict=validate_feed)
                        print("After %s training step(s), validation accuracy = %g" % (global_step, accuracy_score))
                    else:
                        print('No checkpoint file found')
                        return
                time.sleep(100)
        return y,accuracy_score

In [2]:
deepDNN = DeepNN()

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("../data/MNIST_data", one_hot=True)

deepDNN.fit(mnist.train)
y_predict, accuracy_score=deepDNN.predict(mnist.validation)
print("y_predict ", y_predict)
print("accuracy_score",accuracy_score)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
After 1 training step(s), loss on training batch is 2.74457.
After 1001 training step(s), loss on training batch is 0.246995.
After 2001 training step(s), loss on training batch is 0.152804.
After 

(<tf.Tensor 'layer2/add:0' shape=(?, 10) dtype=float32>, 0.9846)